In [ ]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [ ]:
# Get images
image = img_to_array(load_img('woman.png'))
image = np.array(image, dtype=float)

In [ ]:
X = rgb2lab(1.0/255*image)[:,:,0]
Y = rgb2lab(1.0/255*image)[:,:,1:]
Y /= 128
X = X.reshape(1, 400, 400, 1)
Y = Y.reshape(1, 400, 400, 2)

In [ ]:
def conv_stack(filters, d, strides):
    for i in strides:
        model.add(Conv2D(filters, (3, 3), strides=i, activation='relu', dilation_rate=d, padding='same'))

def upsampling_stack(filters):
    for i in filters:
        model.add(Conv2DTranspose(i, (3, 3), strides=(2, 2), activation='relu', padding='same'))
        conv_stack(i, 1, [1, 1])

model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))

conv_stack(8, 1, [1, 2])
conv_stack(16, 1, [1, 2, 2])
conv_stack(32, 2, [1, 1, 1, 1, 1, 1])
upsampling_stack([32, 16, 8])
conv_stack(4, 1, [1, 1, 1])
conv_stack(2, 1, [1])


In [ ]:
# Finish model
model.compile(optimizer='rmsprop',
			loss='mse')

In [ ]:
model.fit(x=X, 
	y=Y,
	batch_size=1,
	epochs=1000)

In [ ]:
print(model.evaluate(X, Y, batch_size=1))
image = img_to_array(load_img('man.png'))
image = np.array(image, dtype=float)
X = rgb2lab(1.0/255*image)[:,:,0]
X = X.reshape(1, 400, 400, 1)

output = model.predict(X)
output *= 128
# Output colorizations
cur = np.zeros((400, 400, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]
imsave("img_result.png", lab2rgb(cur))
imsave("img_gray_version.png", rgb2gray(lab2rgb(cur)))